In [259]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [260]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "pesel",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [261]:
employees_df

,id,imię,nazwisko,pesel,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Anna,Moszyk,82122536825,Wrocław,ul. 400-lecia Bitwy pod Byczyną 43,48907681210,anna.moszyk@kuba.pl,2014-01-01,None,menadżer,70.5
1,2,Mateusz,Hryhorenko,90111107374,Wrocław,ul. Malownicza 45/16,48293869570,mateusz.hryhorenko@kuba.pl,2014-01-01,None,sprzedawca,49.9
2,3,Joanna,Galor,86022705688,Wrocław,ul. Lipowa 7/11,48328082490,joanna.galor@kuba.pl,2014-01-01,2016-07-01,sprzedawca,53.5
5,4,Adrian,Skindzier,61101248717,Bydgoszcz,ul. Myśliwska 84/44,48776067009,adrian.skindzier@kuba.pl,2014-01-01,None,mechanik,49.8
15,5,Waldemar,Kończewski,68111700132,Wrocław,ul. Kazimierza 7,48035608602,waldemar.konczewski@kuba.pl,2014-01-01,None,informatyk,45.8
3,6,Krzysztof,Krzyżanowski,95032281037,Wrocław,ul. Jana Vermeera 43/29,48988505686,krzysztof.krzyzanowski@kuba.pl,2015-02-11,None,sprzedawca,58.1
4,7,Stanisław,Muchowski,65010130976,Wrocław,ul. Łąkowa 83,48072719932,stanislaw.muchowski@kuba.pl,2015-02-25,None,sprzedawca,45.8
6,8,Jan,Matlak,63011844274,Wrocław,ul. Lisia 45/35,48129554842,jan.matlak@kuba.pl,2015-06-20,None,mechanik,57.6
7,9,Kamil,Długołęcki,84052133378,Wrocław,ul. Brzozowa 65/23,48854835426,kamil.dlugolecki@kuba.pl,2015-06-27,2017-05-04,mechanik,57.7
8,10,Klaudyna,Szyjka,68020335403,Wrocław,ul. Świerkowa 3/9,48533895451,klaudyna.szyjka@kuba.pl,2015-07-21,None,mechanik,47.3


In [262]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [263]:
no_transactions = 2000
begining_buy = 20

dates = random_transaction_dates("2014-01-01", "2020-12-31", no_transactions)

In [264]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [265]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(5*max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            jp += 1
            jc += 1

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            jp += 1
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1

In [266]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [267]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [268]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [269]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [270]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [271]:
year_lims = [1970, 2010]
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])
unique_clients = np.unique(client_ids)
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man(year_lims)
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_lims[0], year_lims[1]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Bożena,Kułakowska,K,09280148295,Wrocław,ul. Przyjazna 43/22,False,48795943548,kulakow_bozena09@gmail.com
1,2,Henryk,Kolaczek,M,72051166736,Wrocław,ul. Spokojna 62/37,True,48465633287,kolaczehenryk@gmail.com
2,3,Grażyna,Fołda,K,78021789217,Wrocław,ul. Zacisze 12/32,True,48011308746,foldgra1978@interia.pl
3,4,Adrian,Frąckowiak,M,05260257737,Wrocław,ul. Miętowa 84,False,48286194594,k6s76g03p@onet.pl
4,5,Paweł,Gardoń,M,94110575318,Wrocław,ul. Bartycka 34/13,False,48713982316,paw-gardo@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1481,1482,Dorota,Piątkowska,K,73061395156,Wrocław,ul. Kajki 27/11,False,48730139366,p865x98ayx7m24y4@onet.pl
1482,1483,Renata,Rusek,K,74092118198,Wasilków,ul. Macieja Rataja 46/17,False,48611641830,renru3095@wp.pl
1483,1484,Maciej,Dymek,M,94042532915,Pobiedziska,ul. Marii Konopnickiej 34/46,False,48126458521,m-dymek@gmail.com
1484,1485,Jaśmina,Malinowska,K,82101743972,Kraków,ul. Przemysłowa 29/14,False,48820175662,jas.malinows1982@gazeta.pl


In [272]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,1,Bożena,Kułakowska,K,09280148295,Wrocław,ul. Przyjazna 43/22,False,48795943548,kulakow_bozena09@gmail.com
1,2,Henryk,Kolaczek,M,72051166736,Wrocław,ul. Spokojna 62/37,True,48465633287,kolaczehenryk@gmail.com
2,3,Grażyna,Fołda,K,78021789217,Wrocław,ul. Zacisze 12/32,True,48011308746,foldgra1978@interia.pl
3,4,Adrian,Frąckowiak,M,05260257737,Wrocław,ul. Miętowa 84,False,48286194594,k6s76g03p@onet.pl
4,5,Paweł,Gardoń,M,94110575318,Wrocław,ul. Bartycka 34/13,False,48713982316,paw-gardo@wp.pl
...,...,...,...,...,...,...,...,...,...,...
1481,1482,Dorota,Piątkowska,K,73061395156,Wrocław,ul. Kajki 27/11,False,48730139366,p865x98ayx7m24y4@onet.pl
1482,1483,Renata,Rusek,K,74092118198,Wasilków,ul. Macieja Rataja 46/17,False,48611641830,renru3095@wp.pl
1483,1484,Maciej,Dymek,M,94042532915,Pobiedziska,ul. Marii Konopnickiej 34/46,False,48126458521,m-dymek@gmail.com
1484,1485,Jaśmina,Malinowska,K,82101743972,Kraków,ul. Przemysłowa 29/14,False,48820175662,jas.malinows1982@gazeta.pl


In [273]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [274]:
np.sum(clients_df['karta_lojalnościowa'])

78

In [275]:
np.sum(pd.isna(clients_df["PESEL"]))

53

# auta

In [276]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [277]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [278]:
df_samochody = create_car_table(max(car_ids))

In [279]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,MINI,Cooper Coupe,4,2006,SUV,False,szary,Benzyna,2.0,122,Manualna,5,5,False
1,2,MINI,Cooper S,4,2010,SUV,False,biały,Benzyna,2.8,184,Manualna,5,5,False
2,3,Hero,Glamour 125,2,2007,,False,niebieski,Benzyna,0.5,66,Manualna,0,1,False
3,4,BMW,116,4,2007,Coupe,False,srebrny,Benzyna,2.2,136,Manualna,5,5,False
4,5,Volkswagen,Touran,4,2003,SUV,False,niebieski,Diesel,1.6,105,Manualna,5,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,960,smart,forTwo,4,1998,Sedan,False,szary,Benzyna,1.0,71,Automatyczna,5,5,False
960,961,Toyota,Aygo,4,2001,Hatchback,False,czerwony,Benzyna,1.0,68,Automatyczna,5,5,False
961,962,Honda,Activa 5G,2,2002,,False,czarny,Benzyna,0.4,62,Automatyczna,0,1,False
962,963,Volkswagen,Tiguan,4,2012,SUV,False,szary,Benzyna,4.9,320,Automatyczna,5,5,True


In [280]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [281]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna', 'Półautomatyczna'], dtype=object)

# Usługi

In [282]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [283]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [284]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [285]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\olkar\AppData\Local\Temp\ipykernel_25004\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2007' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [286]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [287]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-02,34900.0
1,2,2,2,6,2014-01-03,95000.0
2,3,3,3,7,2014-01-04,19800.0
3,4,4,4,6,2014-01-05,39100.0
4,5,5,5,3,2014-01-07,32000.0
...,...,...,...,...,...,...
939,940,961,982,7,2020-12-15,14900.0
940,941,962,1480,3,2020-12-16,13100.0
941,942,963,1481,6,2020-12-18,106900.0
942,943,964,1484,6,2020-12-22,19500.0


In [288]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [289]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [290]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania
0,1,10,21,7,2014-01-23
1,2,18,22,7,2014-01-25
2,3,6,23,7,2014-01-26
3,4,19,24,3,2014-01-26
4,5,14,25,2,2014-01-26
...,...,...,...,...,...
935,936,961,1482,3,2020-12-20
936,937,962,1483,3,2020-12-22
937,938,944,1485,3,2020-12-23
938,939,964,1486,6,2020-12-26


In [291]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    print(b_price)
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

df_sell['cena'] = sell_prices

54400.0
30400.0
65000.0
26900.0
24600.0
60000.0
27700.0
40300.0
31700.0
39400.0
54700.0
65800.0
19800.0
48800.0
34900.0
95000.0
32800.0
39100.0
32000.0
65500.0
93800.0
35500.0
34300.0
35600.0
56300.0
25500.0
33400.0
68900.0
48600.0
80100.0
25000.0
20000.0
34600.0
8800.0
51800.0
36700.0
18100.0
60100.0
26100.0
27900.0
57900.0
43500.0
73600.0
34900.0
56300.0
50900.0
23900.0
68500.0
57300.0
51000.0
28500.0
81300.0
20200.0
61500.0
30300.0
31100.0
23600.0
24400.0
38500.0
28400.0
37600.0
42500.0
62800.0
24900.0
64500.0
43800.0
15000.0
30700.0
55200.0
27700.0
31400.0
25500.0
54600.0
22300.0
31100.0
17700.0
26500.0
20900.0
81000.0
61000.0
21200.0
83300.0
14200.0
74100.0
25100.0
11300.0
65500.0
5400.0
41900.0
19400.0
43500.0
24900.0
49500.0
33800.0
34500.0
59700.0
49700.0
27300.0
40500.0
22100.0
106800.0
8300.0
23000.0
70800.0
41400.0
25900.0
26700.0
57300.0
24900.0
74800.0
33600.0
51100.0
17300.0
16500.0
73300.0
22100.0
68700.0
33900.0
30900.0
33400.0
49100.0
40700.0
63200.0
73300.0
36100.0
45

In [292]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,10,21,7,2014-01-23,67900.0
1,2,18,22,7,2014-01-25,42900.0
2,3,6,23,7,2014-01-26,87400.0
3,4,19,24,3,2014-01-26,32500.0
4,5,14,25,2,2014-01-26,32900.0
...,...,...,...,...,...,...
935,936,961,1482,3,2020-12-20,20800.0
936,937,962,1483,3,2020-12-22,15800.0
937,938,944,1485,3,2020-12-23,68000.0
938,939,964,1486,6,2020-12-26,25000.0


In [293]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

0

In [294]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [295]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(25, array([60, 35, 21], dtype=int64)), (43, array([41, 15], dtype=int64)), (60, array([0], dtype=int64)), (64, array([52], dtype=int64)), (78, array([54], dtype=int64)), (92, array([11], dtype=int64)), (94, array([63], dtype=int64)), (149, array([10], dtype=int64)), (157, array([8], dtype=int64)), (158, array([55], dtype=int64)), (161, array([45, 14], dtype=int64)), (170, array([45], dtype=int64)), (185, array([18], dtype=int64)), (188, array([46], dtype=int64)), (212, array([36], dtype=int64)), (217, array([59, 49], dtype=int64)), (220, array([12], dtype=int64)), (223, array([35], dtype=int64)), (250, array([63, 11,  0], dtype=int64)), (251, array([60], dtype=int64)), (290, array([54], dtype=int64)), (293, array([18], dtype=int64)), (295, array([44], dtype=int64)), (296, array([51,  3], dtype=int64)), (298, array([40], dtype=int64)), (300, array([49, 22], dtype=int64)), (303, array([49], dtype=int64)), (321, array([33], dtype=int64)), (327, array([38], dtype=int64)), (332, array([47

In [296]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [297]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Dźwignia zmiany biegów,Opel Adam,75,1,1
1,Przewód hamulcowy,,50,1,1
2,Reflektor,Opel Adam,58,1,1
3,Wał korbowy,Land Rover Range Rover,2303,2,1
4,Radio,Land Rover Range Rover,252,2,1
...,...,...,...,...,...
171,Klocek hamulcowy,Volkswagen Golf,17,113,4
172,Klocek hamulcowy,smart forTwo,11,114,4
173,Fotel przedni,Hero Splendor Plus materiał,53,115,1
174,Klocek hamulcowy,smart forTwo,11,116,4


# Wyposażenie

In [298]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [299]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Dźwignia zmiany biegów,Opel Adam,75.0
1,2,Przewód hamulcowy,,50.0
2,3,Reflektor,Opel Adam,58.0
3,4,Wał korbowy,Land Rover Range Rover,2303.0
4,5,Radio,Land Rover Range Rover,252.0
...,...,...,...,...
170,133,Światła tylne,Volkswagen Golf,60.0
171,134,Klocek hamulcowy,Volkswagen Golf,17.0
172,135,Klocek hamulcowy,smart forTwo,11.0
173,136,Fotel przedni,Hero Splendor Plus materiał,53.0


In [300]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi - uzupełnienie

In [301]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

df_services['cena'] = np.round(service_prices*rng.uniform(1.1, 1.4), -1).astype(int)

In [302]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,25,49,9,2014-02-27,250
1,2,43,84,4,2014-04-15,3530
2,3,60,114,4,2014-05-28,520
3,4,64,52,9,2014-06-07,440
4,5,78,149,9,2014-07-19,830
...,...,...,...,...,...,...
111,112,942,912,9,2020-11-05,530
112,113,943,96,8,2020-11-05,180
113,114,950,1469,9,2020-11-17,60
114,115,957,1476,10,2020-12-02,70


In [303]:
df_services.to_csv("outputs/usługi.csv", index=False)

# Użyte części - uzupełnienie

In [304]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość']]

In [305]:
used_parts_df

,id,id_usługi,id_części,ilość
0,1,1,1,1
1,2,1,2,1
2,3,1,3,1
3,4,2,4,1
4,5,2,5,1
...,...,...,...,...
171,172,113,134,4
172,173,114,135,4
173,174,115,136,1
174,175,116,135,4


In [306]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [307]:
def get_service_date(part_id):
    
    index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
    service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
    return service_date
    

buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'data_zakupu', 'ilość', 'data_zaksięgowania'])

for i in range(len(buy_parts_quantity)):
    part_id = buy_parts_quantity['id_części'][i]
    quantity = buy_parts_quantity['ilość'][i]
    price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
    if price > 1000:
        quantity += rng.random() < 0.5
    elif price > 500:
        quantity += rng.choice([0, 1, 2], p=[0.2, 0.5, 0.3])
    else:
        quantity += rng.binomial(25, 0.15)
    total_cost = np.floor(quantity * price * rng.uniform(0.75, 0.95))
    service_date = get_service_date(part_id)

    ##### DO DODANIA
    # generowanie daty zakupu części na podstawie purchase_date
    # generowanie daty zaksięgowania transakcji na podstawie purchase_date
    #####
    
    # row = [i+1, part_id, total_cost, purchase_date, quantity, posting_date]
    # parts_purchase_pd.loc[i] = row


In [308]:
# parts_purchase_pd